In [1]:
import numpy as np 
import pandas as pd
# import os
# import matplotlib.pyplot as plt
# from PIL import Image, ImageDraw, ImageEnhance
# from tqdm.notebook import tqdm
from tqdm import tqdm

# import cv2
# import re
# import time

import sys
sys.path.append('../')
from retinanet import coco_eval
from retinanet import csv_eval
from retinanet import model
from retinanet import paps_eval
from retinanet import paps_train
# from retinanet import retina
from retinanet.dataloader import *
from retinanet.anchors import Anchors
from retinanet.losses import *
from retinanet.scheduler import *
from retinanet.parallel import DataParallelModel, DataParallelCriterion

# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder

#Torch
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from torch.optim import Adam, lr_scheduler
import torch.optim as optim


In [2]:
import torchvision.models as models
resnet101 = models.resnet101(progress=False, pretrained=True)

In [3]:
# device = torch.device('cpu')
device = torch.device('cuda')
retinanet = model.resnet101(num_classes=2, device=device)
retinanet.load_state_dict(resnet101.state_dict(), strict=False)
retinanet.to(device)

num_features_in of ResidualAfterFPN : 256


ResNet(
  (bn0): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
    

In [4]:

# train_info = np.load('../data/train.npy', allow_pickle=True, encoding='latin1').item()
# train_info

batch_size = 24
dataset_train = PapsDataset('../data/', set_name='train_2class',
                            transform=train_transforms)

train_data_loader = DataLoader(
    dataset_train,
    batch_size=batch_size,
    shuffle=True,
    num_workers=12,
    pin_memory=True,
    prefetch_factor=1,
    collate_fn=collate_fn
)

loading annotations into memory...
Done (t=0.27s)
creating index...
index created!


In [5]:
criterion = PapsLoss(device)
criterion = criterion.to(device)
retinanet.training = True

# https://gaussian37.github.io/dl-pytorch-lr_scheduler/
optimizer = optim.Adam(retinanet.parameters(), lr = 1e-8)
scheduler = CosineAnnealingWarmUpRestarts(optimizer, T_0=20, T_mult=2, eta_max=0.0008,  T_up=5, gamma=0.5)
# CosineAnnealingWarmRestarts

In [6]:
saved_dir = '../trained_models/resnet101_320/dynamicRnn_weightCls/'
s_epoch= 161
e_epoch= 240

In [7]:
state = torch.load(saved_dir + 'all_model.pt')
retinanet.load_state_dict(state['state_dict'])
optimizer.load_state_dict(state['optimizer'])
last_loss = state['loss']

In [8]:
paps_train.train_paps(dataloader=train_data_loader, 
                      model=retinanet, 
                      criterion=criterion,
                      saved_dir=saved_dir, 
                      optimizer=optimizer,
                      scheduler=scheduler,
                      device = device,
                      s_epoch= 161,
                      e_epoch= 240,
                      last_loss = last_loss)  

161th epochs loss 0.09179264078967464 lr 7.939132368836219e-05 
best model is saved


  0%|          | 0/620 [00:00<?, ?it/s]                                                                                             

162th epochs loss 0.10177396239044192 lr 0.000160008 


  0%|          | 0/620 [00:00<?, ?it/s]                                                                                             

163th epochs loss 0.114954950610319 lr 0.000320006 


...

197th epochs loss 0.10920042683107731 lr 0.00028135029493194473 


  0%|          | 0/620 [00:00<?, ?it/s]                                                                                              

198th epochs loss 0.10760851428867915 lr 0.00026180685379001764 


 10%|▉         | 60/620 [00:42<05:34,  1.67it/s, avg_loss=0.114, batch_loss=0.192, cls_loss=0.189, lr=0.000242, reg_loss=0.00333]  Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f0833b0d290>
Traceback (most recent call last):
  File "/home/beomgon/anaconda3/envs/pytorch_retina/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1324, in __del__
    self._shutdown_workers()
  File "/home/beomgon/anaconda3/envs/pytorch_retina/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1297, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/home/beomgon/anaconda3/envs/pytorch_retina/lib/python3.7/multiprocessing/process.py", line 140, in join
    res = self._popen.wait(timeout)
  File "/home/beomgon/anaconda3/envs/pytorch_retina/lib/python3.7/multiprocessing/popen_fork.py", line 45, in wait
    if not wait([self.sentinel], timeout):
  File "/home/beomgon/anaconda3/envs/pytorch_retina/lib/python3.7/multiproce

KeyboardInterrupt: 

In [ ]:
# state = {
#     'epoch': epoch,
#     'state_dict': retinanet.state_dict(),
#     'optimizer': optimizer.state_dict(),
#     'loss' : total_loss/(step+1)
# }
# torch.save(state, saved_dir + 'model.pt')

In [ ]:
# state = {
#     'epoch': epoch,
#     'state_dict': retinanet.state
#     _dict(),
#     'optimizer': optimizer.state_dict(),
#     'loss' : total_loss/(step+1)
# }
# torch.save(state, saved_dir + 'all_model.pt')

In [ ]:
# state = torch.load(saved_dir + 'all_model.pt')
# retinanet.load_state_dict(state['state_dict'])
# optimizer.load_state_dict(state['optimizer'])

In [9]:
dataset_val = PapsDataset('../data/', set_name='val_2class',
                            transform=val_transforms)

val_data_loader = DataLoader(
    dataset_val,
    batch_size=1,
    shuffle=False,
    num_workers=4,
    collate_fn=collate_fn
)

loading annotations into memory...
Done (t=0.12s)
creating index...
index created!


In [10]:
paps_eval.evaluate_paps(dataset=dataset_val, 
  dataloader=val_data_loader, 
  model=retinanet, 
  saved_dir=saved_dir, 
  device = device,
  threshold=0.5)  

100%|██████████| 3739/3739 [02:21<00:00, 26.35it/s]


GT_results 17716
pred_results 3623
validation time : 142.1622085571289
Loading and preparing results...
DONE (t=0.01s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.13s)
creating index...
index created!
******************total*********************
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=1.13s).
Accumulating evaluation results...
DONE (t=0.16s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.048
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.116
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.018
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.037
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.135
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.038
 Average Recall     (AR) @[ Io